In [1]:
!nvidia-smi

Thu Nov 21 05:08:41 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    28W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np
import time

In [3]:
print(tf.__version__)
print(tf.test.is_gpu_available())

2.0.0
True


In [4]:
X_train = np.load('Data/X_train.npy', allow_pickle=True)
y_train = np.load('Data/y_train.npy', allow_pickle=True)

In [5]:
X_train.shape, y_train.shape

((6252, 224, 224, 3), (6252,))

In [6]:
X_train.dtype

dtype('float64')

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [8]:
IMG_SIZE = (224, 224)

@tf.function
def format_train_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, IMG_SIZE)
    # return images as float16
    image = tf.cast(image, tf.float16)
    return image, tf.one_hot(label, 5)

In [9]:
BATCH_SIZE = 80
AUTOTUNE = tf.data.experimental.AUTOTUNE
PREFETCH_COUNT = 8

train_dataset = train_dataset.shuffle(1024)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.map(format_train_example,
                                  num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(PREFETCH_COUNT)

In [10]:
def create_model(img_size=(224,224), num_class=5, train_base=True):
    # Accept float16 image inputs
    input_layer = Input(shape=(img_size[0],img_size[1],3), dtype=tf.float16)
    base = ResNet50(input_tensor=input_layer,
                    include_top=False,
                    weights="imagenet")
    base.trainable = train_base
    x = base.output
    x = GlobalAveragePooling2D()(x)
    
    preds = Dense(num_class, activation="softmax")(x)
    return Model(inputs=input_layer, outputs=preds)

In [11]:
model = create_model()
opt = Adam()
model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [12]:
# Kickstart model training
start = time.time()
model.fit(train_dataset, 
         steps_per_epoch=X_train.shape[0]//BATCH_SIZE,
         epochs=5)
print("It took {} seconds".format(time.time() - start))

Train for 78 steps
Epoch 1/5
78/78 [==============================] - 78s 1s/step - loss: 0.5782 - accuracy: 0.6389
Epoch 2/5
78/78 [==============================] - 59s 754ms/step - loss: 0.7219 - accuracy: 0.5683
Epoch 3/5
78/78 [==============================] - 61s 777ms/step - loss: 0.9141 - accuracy: 0.4896
Epoch 4/5
78/78 [==============================] - 61s 779ms/step - loss: 0.9689 - accuracy: 0.4644
Epoch 5/5
78/78 [==============================] - 61s 787ms/step - loss: 1.0511 - accuracy: 0.4096
It took 319.6887090206146 seconds
